In [1]:
import sys
sys.path.append('../code/')


from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import ShuffleSplit, GridSearchCV, train_test_split

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#from data_handling.handle_data import *
from data_handling.handle_data import get_anime,get_users_ratings
from data_handling.handle_data import get_virgin_data, drop_users_with_too_many_items
from data_handling.handle_data import replace_rating_with_0_and_1
from data_handling.handle_data import get_titles,get_my_ratings
from data_handling.handle_data import drop_users_with_high_proportion_of_ratings
from data_handling.handle_data import transform_ratings


In [3]:
from sklearn.tree import export_graphviz
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, MinMaxScaler, RobustScaler

In [224]:
'''
def transform_ratings(ratings_got):
    ratings = ratings_got.copy(deep=True)
    ratings = ratings.replace([1,2],1)
    #ratings = ratings.replace([3,4],2)
    ratings = ratings.replace([3,4],1)
    #ratings = ratings.replace([5,6],3)
    ratings = ratings.replace([5,6,7],3)
    ratings = ratings.replace([8,9,10],5)
    #ratings = ratings.replace([7,8],4)
    #ratings = ratings.replace([9,10],5)
    return ratings'''

In [225]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

In [226]:
from Recommender.Algorithm import Recommender

In [228]:
K_ECCHI = 3 #best 3 ?15
K_HENTAI = 3 #best 3 ?15

MIN_WACTHED = 14

K_BIG_ITEMS = 1500 # best 1000 ?1500

POSITIVE_RATE_PROPORTION_THRESHOLD = 0.89 #best 0.9 ?1
POSITIVE_IDX=[5] # best 5
# THOUGHT: maybe use 3 classes?

MIN_ITEMS = 15 # best 15
CORRELATION_THRESHOLD = 0.5 #best 0.5
K_USERS_POS = 10# best 20 ?22
K_USERS_NEG = 4# best 15
#best RandomForestClassifier

In [229]:
#PATH_TO_ANIME_DATA = "../data/anime/anime_info.csv"
#anime = pd.read_csv(PATH_TO_ANIME_DATA,index_col=0)
anime = get_anime()

In [231]:
my_ratings = get_my_ratings()

In [232]:
data = get_users_ratings(min_watched=MIN_WACTHED)


In [233]:
data = get_virgin_data(data,anime,K_ECCHI,K_HENTAI)

In [234]:
data.rating = transform_ratings(data.rating)
my_ratings.rating = transform_ratings(my_ratings.rating)

In [235]:
data = drop_users_with_too_many_items(data,K_BIG_ITEMS)

In [236]:

#data = drop_users_with_high_proportion_of_ratings(data,POSITIVE_RATE_PROPORTION_THRESHOLD, POSITIVE_IDX)
data

,user,item,rating
0,0,21,5.0
1,0,59,3.0
2,0,74,3.0
3,0,120,3.0
4,0,178,3.0
...,...,...,...
46358125,283043,4415,5.0
46358126,283043,5940,5.0
46358127,283043,6500,5.0
46358128,283043,7058,5.0


In [237]:
anime.score.unique()

array(['high_rate', 'highest_rate', 'mean_rate', 'middle_rate',
       'low_middle_rate', 'low_rate'], dtype=object)

In [238]:
categories = ['low_rate','low_middle_rate', 'middle_rate','mean_rate','high_rate','highest_rate']

In [239]:
anime_dropped = anime.drop(columns=['Year','score'])

In [240]:
my_rate_anime_combine = my_ratings.set_index('item').join(anime_dropped.iloc[:,4:])
my_rate_anime_combine

,rating,Movie,Music,ONA,OVA,Special,TV,1,1_6,6_11,...,psychological,police,vampire,military,thriller,yaoi,cars,shoujo ai,dementia,yuri
item,,,,,,,,,,,,,,,,,,,,,
7193,5.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9074,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7311,5.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
339,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
164,3.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28223,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2167,5.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29803,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [241]:
NA_indicies = my_rate_anime_combine.iloc[:,1:].isna().all(axis=1)[my_rate_anime_combine.iloc[:,1:].isna().all(axis=1) > 0].index

In [242]:
my_rate_anime_combine = my_rate_anime_combine.drop(index=NA_indicies)
#Full_train

In [243]:
idx = my_ratings.set_index('item').index


In [244]:
pivot = data.loc[data.item.isin(idx)].pivot(columns='user',index='item').rating

In [245]:
#corr_piv = pivot.loc[pivot.index.isin(my_ratings.set_index('item').index)].dropna(how='all')
corr_piv = pivot

In [246]:
users = corr_piv.count()[corr_piv.count() > MIN_ITEMS].index

In [247]:
corr = pivot[users].corrwith(my_ratings.set_index('item').rating)

In [248]:
corr = corr.dropna().sort_values(ascending=False)
#corr

In [249]:
corr

user
173241    0.724363
114595    0.673535
269552    0.636655
174694    0.632456
130707    0.630641
            ...   
246445   -0.358057
33214    -0.382417
187313   -0.382982
127946   -0.427940
244888   -0.485714
Length: 276, dtype: float64

In [250]:
def find_besk_k_users(df, model, corr, k_pos, k_neg, users_only=False):
    mse = float('inf')
    best_pos = 0
    best_neg = 0
    pos_cor = corr[corr > CORRELATION_THRESHOLD]
    neg_cor = corr[corr < -CORRELATION_THRESHOLD].sort_values()
    
    y = df.iloc[:,0]
    for i in range(0,k_pos+1):
        print(i)
        for j in range(0, k_neg+1):
            users_pos = pos_cor.iloc[:i].index
            users_neg =  neg_cor.iloc[:j].index
            users = users_neg.union(users_pos)
            if len(users) == 0 and users_only:
                continue
            X_users_only = df.iloc[:, 57:][users]
            if users_only:
                y_preds = cross_val_predict(model, X_users_only, y, cv=5)
            else:
                X_full = df.iloc[:,1:57].join(X_users_only) #  1 is skip rating
                y_preds = cross_val_predict(model, X_full, y, cv=5)
                
            mse_new = mean_squared_error(y_preds, y)
            if mse > mse_new:
                mse = mse_new
                best_pos = i
                best_neg = j
    print(mse)
    return mse, best_pos, best_neg

In [251]:
df = my_rate_anime_combine.join(pivot.fillna(0))
drop_idx = df.iloc[:,58:].isna().all(axis=1)[df.iloc[:,58:].isna().all(axis=1) > 0].index
df = df.drop(index=drop_idx)

In [252]:
#mse, bp, bn = find_besk_k_users(df, RandomForestClassifier(random_state=42),corr,25,25,True)

In [253]:
users_pos = corr[corr > CORRELATION_THRESHOLD].iloc[:K_USERS_POS].index
users_neg =  corr[corr < -CORRELATION_THRESHOLD].sort_values().iloc[:K_USERS_NEG].index
users = users_neg.union(users_pos)

In [254]:
len(users)

10

In [255]:
#Full_train = Full_train.join(pivot.fillna(-1))
Full_train = my_rate_anime_combine.join(pivot[users].fillna(0))

In [256]:
Full_train.shape

(61, 67)

In [257]:
drop_idx = Full_train.iloc[:,58:].isna().all(axis=1)[Full_train.iloc[:,58:].isna().all(axis=1) > 0].index

In [258]:
Full_train = Full_train.drop(index=drop_idx)

In [259]:
y_full = Full_train.rating
X_full = Full_train.iloc[:,1:]

In [260]:
X_users_only = X_full.iloc[:,56:] # not bad with best

In [261]:
x_train, x_valid, y_train, y_valid = train_test_split(X_full, y_full, test_size=7,random_state=1)

# super cool model

In [267]:
model = KNeighborsClassifier(n_neighbors=3)

In [268]:
total_preds = cross_val_predict(model, X_users_only, y_full, cv=5) # X_full

In [269]:
confusion_matrix(total_preds, y_full).T

array([[15,  0,  5],
       [11,  8,  2],
       [14,  2,  4]])

In [49]:
model.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=10, weights='distance')

In [50]:
model.score(x_train, y_train)

1.0

In [54]:
preds=model.predict(x_valid)

In [52]:
y_valid

item
6351     9.0
27833    8.0
7311     9.0
23273    7.0
16662    7.0
28171    3.0
30276    7.0
Name: rating, dtype: float64

In [143]:
le = LinearRegression()
le.fit(x_train, y_train)
le.score(x_train, y_train)

0.8024313658595124

In [144]:
preds = le.predict(x_valid)
preds

array([9.06957646, 0.89441578, 0.81476921, 2.48611653, 3.50789762,
       1.22456879, 0.18639908])

In [145]:
le.score(x_valid, y_valid)

-9.233519360040837

In [156]:
score = cross_val_score(le, X_users_only, y_full, cv=5) # X_full
score.mean()

0.24615384615384617

In [150]:
y_preds = cross_val_predict(le, X_users_only, y_full, cv=5) # X_users_only
y_preds

array([3., 3., 3., 3., 3., 3., 4., 3., 4., 3., 3., 1., 3., 3., 5., 4., 5.,
       4., 4., 3., 4., 1., 1., 4., 4., 3., 4., 3., 4., 3., 3., 2., 1., 1.,
       5., 3., 3., 1., 3., 2., 4., 3., 3., 4., 4., 3., 4., 4., 1., 3., 1.,
       3., 4., 4., 4., 3., 2., 4., 4., 1., 2.])

In [151]:
mean_squared_error(y_preds, y_full)

1.7868852459016393

In [152]:
y_full.values

array([4., 1., 5., 1., 4., 3., 5., 1., 5., 4., 3., 1., 4., 3., 4., 3., 4.,
       5., 3., 5., 4., 2., 1., 5., 5., 5., 2., 3., 1., 2., 3., 4., 1., 1.,
       4., 1., 3., 3., 4., 1., 5., 4., 4., 5., 3., 1., 4., 3., 1., 4., 1.,
       4., 5., 1., 4., 2., 4., 5., 1., 2., 4.])

In [153]:
res = y_full.to_frame().reset_index().join(pd.Series(y_preds, name='le'))

In [154]:
res.tail(11)

,item,rating,le
50,28907,1.0,1.0
51,16662,4.0,3.0
52,4382,5.0,4.0
53,7596,1.0,4.0
54,1943,4.0,4.0
55,323,2.0,3.0
56,28223,4.0,2.0
57,2167,5.0,4.0
58,29803,1.0,4.0
59,6547,2.0,1.0


In [51]:
for i, preds in enumerate(y_preds):
    if preds > 10:
        y_preds[i] = 10
    elif preds < 1:
        y_preds[i] = 1

In [52]:
y_preds

array([3.20558879, 2.6191147 , 5.90921403, 2.74802197, 4.68719148,
       2.3864028 , 5.61786159, 2.90833986, 2.2922832 , 3.46718121,
       3.13127656, 4.50736127, 1.        , 3.93848383, 4.39740942,
       2.92170468, 1.28153037, 6.24819852, 3.70014654, 1.35182049,
       1.        , 6.53727472, 2.15184464, 4.27018812, 4.93536044,
       2.80046681, 1.53446872, 2.80046681, 3.40597311, 2.80046681,
       1.52234905, 3.00388644, 1.        , 1.        , 2.60349702,
       2.80046681, 1.96518031, 3.04108231, 2.50711228, 2.74284922,
       4.16455803, 2.84339575, 2.50711228, 2.50711228, 2.50711228,
       2.50711228, 5.67855948, 5.19650903, 1.        , 1.        ,
       2.94874173, 2.21275581, 5.39589742, 2.77516119, 2.60151687,
       4.18163386, 7.20934473, 4.29209124, 2.4863986 , 5.92253653,
       5.99832332])

In [160]:
clf = RandomForestClassifier(random_state=42)
clf.fit(x_train, y_train)
clf.score(x_train,y_train)

1.0

In [161]:
clf.score(x_valid,y_valid)

0.42857142857142855

In [162]:
preds = clf.predict(x_valid)
preds

array([5., 1., 3., 4., 3., 4., 4.])

In [163]:
y_valid.values

array([5., 4., 5., 4., 4., 2., 4.])

In [164]:
score = cross_val_score(clf, X_users_only, y_full, cv=5) # X_full
score.mean()

0.24615384615384617

In [165]:
score

array([0.23076923, 0.25      , 0.33333333, 0.25      , 0.16666667])

In [166]:
y_preds = cross_val_predict(clf, X_users_only, y_full, cv=5) # X_users_only

In [167]:
matrix = confusion_matrix(y_full, y_preds)
matrix

array([[6, 1, 5, 3, 0],
       [2, 0, 2, 1, 0],
       [1, 0, 6, 4, 0],
       [0, 2, 9, 3, 4],
       [0, 0, 3, 9, 0]])

In [ ]:
model = LogisticRegression(C=0.7, random_state=42,max_iter=500, class_weight={1:10, 3:1,4:1,5:1})
#model = clf
y_preds = cross_val_predict(model, X_full, y_full, cv=5) # X_users_only
matrix = confusion_matrix(y_full, y_preds)
matrix

In [ ]:
for i in range(len(matrix)):
    print(matrix[i].sum())

In [ ]:
model.fit(X_full, y_full)

In [ ]:
bad_rates_count_as_good = matrix[0][2] + matrix[0][3] + matrix[1][2] + matrix[1][3]
good_rates_count_as_bad = matrix[2][0] + matrix[2][1] + matrix[3][0] + matrix[3][1]
bad_rates_true = matrix[0][0] + matrix[0][1] + matrix[1][0] + matrix[1][1]
good_rates_true = matrix[2][2] + matrix[2][3] + matrix[3][2] + matrix[3][3]
print(bad_rates_count_as_good) # 5
print(good_rates_count_as_bad) # 9
print(bad_rates_true) # 36
print(good_rates_true) # 11

In [ ]:
#for i, val in enumerate(y_preds):
#    print(y_preds[i],' ; ',y_full.values[i])

# What about dummies predictors?

In [ ]:
from sklearn.dummy import DummyClassifier

In [ ]:
dummy = DummyClassifier(strategy='most_frequent')

In [ ]:
dummy_score = cross_val_score(dummy, X_full, y_full, cv=5)
dummy_score.mean()

In [ ]:
dummy_score

In [ ]:
dummy.fit(x_train,y_train)

In [ ]:
dummy.predict(x_valid)

In [ ]:
dummy.score(x_train, y_train)

In [ ]:
dummy.score(x_valid, y_valid)

In [ ]:
y_full.unique()

In [ ]:
#fig, ax = plt.subplots(figsize=(50, 50))  # whatever size you want
#plot_tree(clf, filled=True,fontsize=30, ax=ax, feature_names=Full_train.columns)

In [ ]:
#fn=X_full.columns
#cn=y_full.unique()
#fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (14,19), dpi=800)
#plot_tree(clf.estimators_[1],
#               feature_names = fn, 
#               #class_names=cn,
#               filled = True);

In [ ]:
#fig, ax = plt.subplots(figsize=(50, 50))  # whatever size you want
#plot_tree(clf, filled=True,fontsize=30, ax=ax, feature_names=Full_train.columns)

In [ ]:
#grid = GridSearchCV(clf, parameters)

In [ ]:
#grid.fit(X_full, y_full)

In [ ]:
#clf = RandomForestRegressor(max_depth=5, n_estimators=150, n_jobs=8,random_state=42)

In [ ]:
#fig, ax = plt.subplots(figsize=(10, 30))  # whatever size you want
#plot_tree(clf, filled=True,fontsize=20, ax=ax,class_names=cn,feature_names=X_full.columns)


# Recs

In [181]:
pivot_full = data[data.user.isin(users)].pivot(columns='user', index='item').rating

In [182]:
#X = anime.iloc[:,4:].join(pivot.fillna(-1))
#X = anime.iloc[:,4:].join(pivot_full[users].fillna(0))
X = anime_dropped.iloc[:,4:].join(pivot_full.fillna(0))
#X = anime.iloc[:,4:]

In [183]:
drop_idx = X.iloc[:,58:].isna().all(axis=1)[X.iloc[:,58:].isna().all(axis=1) > 0].index

In [184]:
len(drop_idx)

12959

In [185]:
#preds = clf.predict(pivot_full.fillna(0))
#model = le #X_full  X_users_only
model.fit(X_users_only, y_full)
preds = model.predict(X.iloc[:,56:].drop(index=drop_idx))

In [186]:
recs = pd.Series(preds,index=X.drop(index=drop_idx).index, name='preds')

In [187]:
recs.sort_values(ascending=False)

item
1689     5.0
4059     5.0
7674     5.0
7724     5.0
1840     5.0
        ... 
34009    1.0
5251     1.0
28755    1.0
10370    1.0
11013    1.0
Name: preds, Length: 717, dtype: float64

In [188]:
items=recs.sort_values(ascending=False)

In [189]:
preds = anime[['title']].join(items).dropna().sort_values('preds', ascending=False)

In [190]:
clear_preds = preds.loc[~preds.index.isin(my_ratings.set_index('item').index)]

In [191]:
clear_preds.head(50) #578

,title,preds
item,,
2924,ef: A Tale of Memories.,5.0
14131,Girls & Panzer,5.0
97,Last Exile,5.0
15117,Kami nomi zo Shiru Sekai: Tenri-hen,5.0
7724,Shiki,5.0
11111,Another,5.0
6702,Fairy Tail,5.0
4789,ef: A Tale of Melodies.,5.0
3712,Zero no Tsukaima: Princesses no Rondo,5.0


In [192]:
preds[preds.title.str.contains('Re:Zero')]

,title,preds
item,,
31240,Re:Zero kara Hajimeru Isekai Seikatsu,3.0


In [104]:
#clear_preds[clear_preds.preds > 3.5].head(50)

In [195]:
preds[preds.title.str.contains('One')]

,title,preds
item,,
6280,Cat Shit One,4.0
30276,One Punch Man,4.0
31704,One Punch Man: Road to Hero,3.0
5040,One Outs,1.0
31772,One Punch Man Specials,1.0
15323,One Piece: Episode of Nami - Koukaishi no Nami...,1.0
13283,One Off,1.0
34392,One Room,1.0
4155,One Piece Film: Strong World,1.0
